In [1]:
print("""
@File         : CH13.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-08-31 10:07:21
@Email        : cuixuanstephen@gmail.com
@Description  : 处理 Excel 电子表格
""")


@File         : CH13.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-08-31 10:07:21
@Email        : cuixuanstephen@gmail.com
@Description  : 处理 Excel 电子表格



In [2]:
%cd ../

d:\Python-3\ABSP2E


In [3]:
import openpyxl

## 读取 Excel 文档

### 用 openpyxl 模块打开 Excel 文档

`openpyxl.load_workbook()` 函数接收文件名，并返回一个 `Workbook` 数据类型的值。这个 `Workbook` 对象代表这个 Excel 文件，这有点儿类似于 `File` 对象代表一个打开的文本文件。

In [4]:
wb = openpyxl.load_workbook('sources/example.xlsx')
type(wb)

openpyxl.workbook.workbook.Workbook

### 从工作簿中取得工作表

访问 `sheetnames` 属性可以取得工作簿中所有表名的列表。

In [5]:
wb.sheetnames

['Sheet1', 'Sheet2', 'Sheet3']

In [6]:
sheet = wb['Sheet3']
sheet

<Worksheet "Sheet3">

In [7]:
type(sheet)

openpyxl.worksheet.worksheet.Worksheet

In [8]:
sheet.title

'Sheet3'

In [9]:
another_sheet = wb.active
another_sheet

<Worksheet "Sheet1">

每个表由一个 `Worksheet` 对象表示，取得它的方法是使用带方括号的工作表名称字符串，这和取得字典的键一样。最后，可以使用 `Workbook` 对象的 `active` 属性来取得工作簿的活动表。在取得 `Worksheet` 对象后，可以通过 `title` 属性取得它的名称。

### 从表中取得单元格

有了 `Worksheet` 对象后，就可以按名字访问 `Cell` 对象。

In [10]:
sheet = wb['Sheet1']
sheet['A1'].value

datetime.datetime(2015, 4, 5, 13, 34, 2)

In [11]:
c = sheet['B1']
c.value

'Apples'

In [12]:
f'Row {c.row}, Column {c.column} is {c.value}'

'Row 1, Column 2 is Apples'

In [13]:
f'Cell {c.coordinate} is {c.value}'

'Cell B1 is Apples'

In [14]:
sheet['C1'].value

73

`Cell` 对象有一个 value 属性，它包含这个单元格中保存的值。`Cell` 对象也有 `row`、`column` 和 `coordinate` 属性，可以提供该单元格的位置信息。

openpyxl 将自动解释列中的日期，将它们返回为 `datetime` 值，而不是字符串。

用字母来指定列，这在程序中可能有点儿奇怪，特别是在 Z 列之后，列开始使用两个字母：AA、AB、AC 等表示。作为替代，在调用表的 `cell()` 方法时，传入整数作为该方法的 `row` 和 `column` 关键字参数，这样也可以得到一个单元格。**第一行或第一列的整数是 1，不是 0。**

In [15]:
sheet.cell(row=1, column=2)

<Cell 'Sheet1'.B1>

In [16]:
sheet.cell(row=1, column=2).value

'Apples'

In [17]:
for i in range(1, 8, 2):
    print(i, sheet.cell(row=i, column=2).value)

1 Apples
3 Pears
5 Apples
7 Strawberries


可以通过 `Worksheet` 对象的 `max_row` 和 `max_column` 属性来确定表的大小。

In [18]:
sheet = wb['Sheet1']
sheet.max_row # Get the highest row number.

7

In [19]:
sheet.max_column

3

请注意，`max_column` 属性是一个整数。

### 列字母和数字之间的转换
要从字母转换到数字，就调用 `openpyxl.utils.column_index_from_string()` 函数。要从数字转换到字母，就调用 `openpyxl.utils.get_column_letter()` 函数。

In [20]:
from openpyxl.utils import get_column_letter, column_index_from_string

In [21]:
get_column_letter(1)

'A'

In [22]:
get_column_letter(2)

'B'

In [23]:
get_column_letter(27)

'AA'

In [24]:
get_column_letter(900)

'AHP'

In [25]:
wb = openpyxl.load_workbook('sources/example.xlsx')
sheet = wb['Sheet1']
get_column_letter(sheet.max_column)

'C'

In [26]:
column_index_from_string("A")

1

In [27]:
column_index_from_string("AA")

27

### 从表中取得行和列

可以将 `Worksheet` 对象切片，取得电子表格中一行、一列或一个矩形区域中的所有 `Cell` 对象。然后可以循环遍历这个切片中的所有单元格。

In [29]:
sheet = wb['Sheet1']
tuple(sheet['A1': 'C3']) # Get all cells from A1 to C3.

((<Cell 'Sheet1'.A1>, <Cell 'Sheet1'.B1>, <Cell 'Sheet1'.C1>),
 (<Cell 'Sheet1'.A2>, <Cell 'Sheet1'.B2>, <Cell 'Sheet1'.C2>),
 (<Cell 'Sheet1'.A3>, <Cell 'Sheet1'.B3>, <Cell 'Sheet1'.C3>))

In [30]:
for row in sheet['A1': 'C3']:
    for cell in row:
        print(cell.coordinate, cell.value)
    print('--- END OF ROW ---')

A1 2015-04-05 13:34:02
B1 Apples
C1 73
--- END OF ROW ---
A2 2015-04-05 03:41:23
B2 Cherries
C2 85
--- END OF ROW ---
A3 2015-04-06 12:46:51
B3 Pears
C3 14
--- END OF ROW ---


这里，我们指明需要从 A1 到 C3 的矩形区域中的 `Cell` 对象，我们还得到了一个 `Generator` 对象，它包含该区域中的 `Cell` 对象。为了弄清楚这个 `Generator` 对象，可以对它使用 `tuple()` 方法以在一个元组中列出它的 `Cell` 对象。

要访问特定行或列的单元格的值，也可以利用 `Worksheet` 对象的 `rows` 和 `columns` 属性。这些属性必须被 `list()` 函数转换为列表，才能使用方括号和索引。

In [32]:
sheet = wb.active
list(sheet.columns)[1]

(<Cell 'Sheet1'.B1>,
 <Cell 'Sheet1'.B2>,
 <Cell 'Sheet1'.B3>,
 <Cell 'Sheet1'.B4>,
 <Cell 'Sheet1'.B5>,
 <Cell 'Sheet1'.B6>,
 <Cell 'Sheet1'.B7>)

In [33]:
for cell in list(sheet.columns)[1]:
    print(cell.value)

Apples
Cherries
Pears
Oranges
Apples
Bananas
Strawberries


利用 `Worksheet` 对象的 `rows` 属性，可以得到一个元组构成的元组。内部的每个元组都代表 1 行，包含该行中的 `Cell` 对象。`columns` 属性也会给你一个元组构成的元组，内部的每个元组都包含 1 列中的 `Cell` 对象。

### 工作簿、工作表、单元格

下面是从电子表格文件中读取单元格涉及的所有函数、方法和数据类型。
1. 导入 `openpyxl` 模块。
2. 调用 `openpyxl.load_workbook()` 函数。
3. 取得 `Workbook` 对象。
4. 使用 `active` 或 `sheetnames` 属性。
5. 取得 `Worksheet` 对象。
6. 使用索引或工作表的 `cell()` 方法，带上 `row` 和 `column` 关键字参数。
7. 取得 `Cell` 对象。
8. 读取 `Cell` 对象的 `value` 属性。

## [项目：从电子表格中读取数据]()